## Задача. Построить модель прогнозирования факта сдачи теста

<img src="image/image1.png" alt="Drawing" style="width: 380px;" align="left"/> <br />

### Загружаем библиотеки

In [1]:
import itertools
import numpy as np
import pandas as pd
import scipy

from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline 

### Загружаем данные

In [2]:
df = pd.read_csv('dataset.csv', sep=';')
print(df.shape)
df.head()

(2133, 12)


,Unnamed: 0,Unnamed: 0.1,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch,posttest,target
0,0,0,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,72.0,1.0
1,1,1,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,79.0,0.0
2,2,2,ANKYI,Urban,Non-public,6OL,Standard,NaN,Male,Does not qualify,76.0,1.0
3,3,3,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,77.0,1.0
4,4,4,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,Does not qualify,76.0,0.0


### Описание полей
* **school** - Код школы, в которой учится ученик  
* **school_setting** - Тип района в котором находится школа  
* **school_type** - Частная или нет школа  
* **classroom** - внутренний код класса, в котором проиходят занятия  
* **teaching_method** - является ли обучение экспериментальным  
* **n_student** - возраст ученика  
* **gender** - пол ученика  
* **lunch** - Является ли ученик слабозащищенной категорией населения  
* **posttest** - балл по тесту
* **target** - сдал или нет. 1- сдал.

### Разобьем выборку на train / test

In [3]:
X = df.drop(['posttest'], axis=1)
X = X.drop(['target'], axis=1)
y = df['posttest']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)
X_train.shape, X_test.shape

((1599, 10), (534, 10))

### Заменим пропуски в данных

In [4]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

-----------

В качестве целевой переменной  будем рассматривать балл по тесту.  <br />
Построим линейную регрессию.

### Заинжинирим новую переменную
пол*возраст
для каждой комбинации посчитаем weight of evidence и будем его использовать в модели

In [5]:
for i in df.gender.unique():
    for j in df.n_student.unique():
        l=df.loc[(df.gender==i) & (df.n_student>=j)]
        woe=l.target.sum()/(l.target.count()-df.target.mean()-1)
        df.loc[(df.gender==i) & (df.n_student>=j), 'woe_agegender']=woe

### Дальше накинем one-hot encoding

In [6]:
class LinearRegressionEncoder(BaseEstimator):
    def __init__(self):
        self.linreg = LinearRegression()
        self.encoders = {}
    
    
    def fit(self, X, y):
        X_enc = X.copy()
        categorical_features = X.columns[X.dtypes == 'object']
        
        for col in categorical_features:
            le = LabelEncoder()
            X_enc[col] = le.fit_transform(X[col])
            self.encoders[col] = le
            
        self.linreg.fit(X_enc, y)

        # считаем значимость
        y_pred = self.linreg.predict(X_enc)
        sse = np.sum((y_pred - y) ** 2, axis=0) / float(X_enc.shape[0] - X_enc.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.pinv(np.dot(X_enc.T, X_enc))))])

        self.t = self.linreg.coef_ / se
        self.p = 2 * (1 - scipy.stats.t.cdf(np.abs(self.t), y.shape[0] - X_enc.shape[1]))
        
        return self
    
    def predict(self, X):
        X_enc = X.copy()
        categorical_features = X.columns[X.dtypes == 'object']
        
        for col in categorical_features:
            X_enc[col] = self.encoders[col].transform(X[col])
            
        y_pred = self.linreg.predict(X_enc)
        
        return y_pred
    
    def score(self, X, y):        
        
        y_pred = self.predict(X)
        return mean_absolute_error(y, y_pred)

In [7]:
df.target.unique()

array([ 1.,  0., nan])

In [8]:
%%time
linreg = LinearRegressionEncoder()
linreg.fit(X_train, y_train)

Wall time: 88 ms


LinearRegressionEncoder()

In [9]:
print('TRAIN MSE:', linreg.score(X_train, y_train))
print('TEST MSE:', linreg.score(X_test, y_test))

TRAIN MSE: 7.4971977724881365
TEST MSE: 7.661175811821424


### Ошибка на тесте не сильно больше чем на трейне. Значит модель отличная

Улучшим модель, исключив из данных незначимые факторы

In [10]:
## здесь дописать
for p_val, factor in itertools.zip_longest(np.array(linreg.p[0]), np.array(X.columns)):
    print(factor, ' - ', round(p_val, 6))

Unnamed: 0  -  1.4e-05
Unnamed: 0.1  -  1.4e-05
school  -  8e-06
school_setting  -  0.005098
school_type  -  1e-06
classroom  -  0.593291
teaching_method  -  0.0
n_student  -  0.0
gender  -  0.044671
lunch  -  0.0


удалим факторы у которых pvalue = 0 и перевзвесим коэффициенты

In [11]:
sign = [f for p, f in itertools.zip_longest(np.array(linreg.p[0]), np.array(X.columns)) if p == 0]
sign

['teaching_method', 'n_student', 'lunch']

In [12]:
%%time
linreg2 = LinearRegressionEncoder()
linreg2.fit(X_train.drop(sign, axis=1), y_train)

Wall time: 10 ms


LinearRegressionEncoder()

In [13]:
print('TRAIN MSE after drop insignificant:', 
      linreg2.score(X_train.drop(sign, axis=1), y_train))
print('TEST MSE after drop insignificant', 
      linreg2.score(X_test.drop(sign, axis=1), y_test))

TRAIN MSE after drop insignificant: 9.948255552267822
TEST MSE after drop insignificant 9.80995812175081


### После удаления получили модель чуть похуже, но в ней оценки коэффициентов несмещены